In [1]:
import pandas as pd
import sys
sys.path.append(r"C:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\myLibraries")

In [2]:
dataset_url="https://drive.google.com/file/d/1Wd0M3qepNF6B4YwFYrpo7CaSERpudAG_/view?usp=share_link"
dataset_url='https://drive.google.com/uc?export=download&id=' + dataset_url.split('/')[-2]
dataset = pd.read_json(dataset_url, lines=True)
dataset = dataset.loc[:, ["text", "summary"]]

In [3]:
subset = dataset.iloc[:2, :]
subset

,text,summary
0,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Science Education Tax Incentive for B...
1,SECTION 1. SHORT TITLE.\n\n This Act may be...,Small Business Expansion and Hiring Act of 201...


### MARScore class creation

In [ ]:
from MARScore.utils import *
import hdbscan
import os

class MARSCore():
    def __init__(self, corpus, model=BertModel.from_pretrained('bert-base-uncased', 
                                                               output_hidden_states=True), 
                               tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')) -> None:
        """
        Constructor of the MARScore class.

        :param1 corpus (list): List of texts to summarize.
        :param2 model (transformer): Transformer model used compute dynamic embeddings.
        :param3 tokenizer (transformer) Transformer used to create token from a plain text. 
        """
        self.corpus = corpus
        self.summary = []
        self.model = model
        self.tokenizer = tokenizer
    
    def compute(self):
        for indiv in self.corpus:
            #creation of embeddings
            o, l = tokenizeCorpus(indiv)
            v = vectorizeCorpus(o)
            v, l = cleanAll(v, l)

            #clusterization
            clusterer = hdbscan.HDBSCAN()
            clusterer.fit(v)
            clabels = clusterer.labels_

            #TF calculation
            tf_values = tf(l)
            clusters_tf_values = clusters_tf(tf_values, l, clabels)

            #ILP computation
            check = to_ilp_format(l, clabels, clusters_tf_values)
            root = get_git_root()
            dirpath = os.path.join(root, "myLibraries\MARScore_output")
            os.system(f'glpsol --tmlim 100 --lp "{os.path.join(dirpath, "ilp_in.ilp")}" -o "{os.path.join(dirpath, "ilp_out.sol")}"')
            selected = readILP()

            #summary construction
            sentences = indiv.split(".")
            sentences.pop()
            sum_sentences = []
            for i, value in enumerate(selected):
                if value == 1:
                    sum_sentences.append(sentences[i]+".")
            self.summary.append(" ".join(sum_sentences))

In [ ]:
mscore = MARSCore(subset["text"])
mscore.compute()

In [ ]:
mscore.summary

### MARScore class test

In [4]:
from MARScore.score import MARSCore

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
ms = MARSCore(subset["text"])
ms.compute()

In [7]:
ms.summary

["SECTION 1. \n\n    This Act may be cited as the ``National Science Education Tax \nIncentive for Businesses Act of 2007''. \n\n    (a) In General. --The term `STEM school contributions' \n        means--\n                    ``(A) STEM property contributions, and\n                    ``(B) STEM service contributions. \n        The determination of the amount of deduction under section 170 \n        for purposes of this paragraph shall be made as if the \n        limitation under section 170(e)(3)(B) applied to all STEM \n        inventory property. \n    ``(d) STEM Teacher Externship Expenses. --The term `STEM teacher externship \n        expenses' means any amount paid or incurred to carry out a STEM \n        externship program of the taxpayer but only to the extent that \n        such amount is attributable to the participation in such \n        program of any eligible STEM teacher, including amounts paid to \n        such a teacher as a stipend while participating in such \n     